In [13]:
import requests
from time import sleep
import random
from bs4 import BeautifulSoup
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import sys
import datetime


df = pd.read_csv("Apartments.csv", encoding="utf-16", delimiter = ";", error_bad_lines = False)
df.head()



### DEALING WITH MISSING DATA

#df[df.isnull().any(axis= 1)]

df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)



### CHECK FOR DUPLICATES

#df.duplicated().any()



### DATA ADJUSTMENTS

# removing spaces
cols = df.columns

for col in df.columns:
    df[f"{col}"] = df[f"{col}"].str.strip()
    
    
# seperate action price
akcija = df[df["price"] == "Stanje"]


# regular price
df = df[(df["price"] != "Po dogovoru") & (df["price"] != "Stanje")]
df.reset_index(inplace = True, drop = True)


# seperate data for price per agreement
po_dogovoru = df[df["price"] == "Po dogovoru"]
po_dogovoru.reset_index(inplace = True, drop = True)


# getting action prices 
prices = []

for apartment in akcija["link"]:
    response = requests.get(apartment, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")
    prices.append(soup.find("p", style = "font-size:25px;font-weight:500;background-color:#9dab50;color:#fff;").text[11:-1].strip())
    
    sleep(random.randint(1, 3))

akcija["price"] = prices


# concat regular and action
df = pd.concat([df, akcija])
df.reset_index(inplace = True, drop = True)     

    
    
### PRICE

# remove currency
df["price"] = df["price"].str[:-2].str.strip()

# remove "."
df["price"] = df["price"].str.replace(".", "")

# change price to numeric
df["price"] = pd.to_numeric(df["price"])



# N OF SQM

# taking just numbers
df["n of sqm"] = df["n of sqm"].str[:2]


# removing non num entries
numbers = [str(i) for i in range(100)]

df = df[df["n of sqm"].isin(numbers)]
df.reset_index(inplace = True, drop = True)


# change to numeric
df["n of sqm"] = pd.to_numeric(df["n of sqm"])


# create price of sqm
df["price_sqm"] = (df["price"]/df["n of sqm"]).astype(int)



# N OF ROOMS

# recode
df["n of rooms"].replace(
    
    {"Trosoban (3)": "3",
     "Dvosoban (2)": "2",
     "Četverosoban (4)": "4",
     "Jednosoban (1)": "1",
     "Petosoban i više": "5",
     "Garsonjera" : "0",
     "Jednoiposoban (1.5)": "1.5",}, inplace = True)

# convert to num
df["n of rooms"] = pd.to_numeric(df["n of rooms"], downcast = "float")



# PUBLISH DATE

# take just date
df["publish date"] = df["publish date"].str[:-8]

# get day month and year (had problem with datetime type so I had to do it this way)
df["day"] = df["publish date"].str[:2]
df["month"] = df["publish date"].str[3:5]
df["year"] = df["publish date"].str[6:-1]

# drop publish date
df.drop("publish date", axis = "columns", inplace = True)


# setting up conditions and filtering data based on them
'''
Conditions:
- price <= 150000
- n of rooms >= 2
- n of sqm >= 30
- location Novo Sarajevo and Centar 
- floor != prizemlje & suteren
''' 
df = df[(df["price"] <= 150000) & 
    (df["n of rooms"] >= 2) & 
    (df["n of sqm"] >= 30) & 
    ((df["location"] == "Sarajevo - Centar") | (df["location"] == "Novo Sarajevo")) & 
    (df["floor"] != "Prizemlje") & 
    (df["floor"] != "Suteren")]


df = df.sort_values(by = ["year", "month", "day"], ascending = False)
df.reset_index(inplace = True, drop = True)

C:\Users\Amar\AppData\Local\Temp\ipykernel_1552\1425867191.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("Apartments.csv", encoding="utf-16", delimiter = ";", error_bad_lines = False)
b'Skipping line 1009: expected 9 fields, saw 10\n'
C:\Users\Amar\AppData\Local\Temp\ipykernel_1552\1425867191.py:81: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["price"] = df["price"].str.replace(".", "")


In [14]:
df

,header,location,price,n of sqm,floor,n of rooms,heating system,link,price_sqm,day,month,year
0,MY SPACE/ Stan/ Velesici/ Muhameda Ef Pandze/ ...,Novo Sarajevo,145000,61,Visoko prizemlje,3.0,Centralno (Plin),https://www.olx.ba/artikal/48576223/my-space-s...,2377,21,06,2022
1,"Dvosoban stan Sarajevo, Centar, Višnjik",Sarajevo - Centar,149000,45,1,2.0,Struja,https://www.olx.ba/artikal/48574330/dvosoban-s...,3311,21,06,2022
2,Dvosoban stan - Centar - Koševsko brdo - 36 m2,Sarajevo - Centar,99500,36,4,2.0,Struja,https://www.olx.ba/artikal/48565044/dvosoban-s...,2763,20,06,2022
3,Dvosoban renoviran stan Sarajevo Centar Bjelav...,Sarajevo - Centar,149000,45,1,2.0,Struja,https://www.olx.ba/artikal/48543663/dvosoban-r...,3311,18,06,2022
4,Stan Grbavica kod soppinga,Novo Sarajevo,135000,38,4,2.0,Centralno (gradsko),https://www.olx.ba/artikal/48526088/stan-grbav...,3552,17,06,2022
5,"Dvosoban stan Socijalno, 41m2",Novo Sarajevo,149000,41,Visoko prizemlje,2.0,Centralno (gradsko),https://www.olx.ba/artikal/48504008/dvosoban-s...,3634,15,06,2022
6,"POSJED PRODAJE:Dvosoban namješten stan, Socijalno",Novo Sarajevo,149000,41,Visoko prizemlje,2.0,Centralno (gradsko),https://www.olx.ba/artikal/48475581/posjed-pro...,3634,13,06,2022
7,"Dvosoban stan, Dobrinja/Sarajevo, 44m2",Novo Sarajevo,149500,44,8,2.0,Centralno (Plin),https://www.olx.ba/artikal/48317448/dvosoban-s...,3397,31,05,2022
8,"AD COMPANY prodaje: Stan, Centar (1107)",Sarajevo - Centar,99500,38,4,2.0,Struja,https://www.olx.ba/artikal/48225707/ad-company...,2618,24,05,2022
9,NN Nekretnine: Dvosoban adaptiran stan - Bjelave,Sarajevo - Centar,148000,42,3,2.0,Plin,https://www.olx.ba/artikal/48040802/nn-nekretn...,3523,09,05,2022


In [ ]:
# email setup

from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

sender_email = input(str("Enter your email: "))
rec_email = input(str("Enter email to send to: "))
password = input(str("Enter pass: "))

msg = MIMEMultipart('alternative')
msg["Subject"] = "Your update on apartments"


html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(df.to_html())


part1 = MIMEText(html, 'html')


msg.attach(part1)


with smtplib.SMTP("smtp-mail.outlook.com", 587) as smtp:
    smtp.ehlo()
    smtp.starttls()
    smtp.ehlo()
    
    smtp.login(sender_email, password)
    print("Login successful")
    
    
    smtp.sendmail(sender_email, rec_email, msg.as_string())
    smtp.quit()
    print("Email sent")